For nominal categorical data
When applied on a column having different categories, it creates separate columns for each category. after that we have to drop the first category column to avoid multicolinerality issue. (n-categories in column ---> n-1 new columns)

Use SciKit-learn library (sklearn) to use One hot encoder

merge categories having less numbers to one to reduce new columns number

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('cars.csv')

In [ ]:
df.head()

brand  km_driven    fuel         owner  selling_price
0   Maruti     145500  Diesel   First Owner         450000
1    Skoda     120000  Diesel  Second Owner         370000
2    Honda     140000  Petrol   Third Owner         158000
3  Hyundai     127000  Diesel   First Owner         225000
4   Maruti     120000  Petrol   First Owner         130000

In [ ]:
df['owner'].value_counts()

First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: owner, dtype: int64

## 1. OneHotEncoding using Pandas

In [ ]:
pd.get_dummies(df,columns=['fuel','owner'])

brand  km_driven  selling_price  fuel_CNG  fuel_Diesel  fuel_LPG  \
0      Maruti     145500         450000         0            1         0   
1       Skoda     120000         370000         0            1         0   
2       Honda     140000         158000         0            0         0   
3     Hyundai     127000         225000         0            1         0   
4      Maruti     120000         130000         0            0         0   
...       ...        ...            ...       ...          ...       ...   
8123  Hyundai     110000         320000         0            0         0   
8124  Hyundai     119000         135000         0            1         0   
8125   Maruti     120000         382000         0            1         0   
8126     Tata      25000         290000         0            1         0   
8127     Tata      25000         290000         0            1         0   

      fuel_Petrol  owner_First Owner  owner_Fourth & Above Owner  \
0               0                  1                           0   
1               0                  0                           0   
2               1                  0                           0   
3               0                  1                           0   
4               1                  1                           0   
...           ...                ...                         ...   
8123            1                  1                           0   
8124            0                  0                           1   
8125            0                  1                           0   
8126            0                  1                           0   
8127            0                  1                           0   

      owner_Second Owner  owner_Test Drive Car  owner_Third Owner  
0                      0                     0                  0  
1                      1                     0                  0  
2                      0                     0                  1  
3                      0                     0                  0  
4                      0                     0                  0  
...                  ...                   ...                ...  
8123                   0                     0                  0  
8124                   0                     0                  0  
8125                   0                     0                  0  
8126                   0                     0                  0  
8127                   0                     0                  0  

[8128 rows x 12 columns]

## 2. K-1 OneHotEncoding

In [ ]:
pd.get_dummies(df,columns=['fuel','owner'],drop_first=True)

brand  km_driven  selling_price  fuel_Diesel  fuel_LPG  fuel_Petrol  \
0      Maruti     145500         450000            1         0            0   
1       Skoda     120000         370000            1         0            0   
2       Honda     140000         158000            0         0            1   
3     Hyundai     127000         225000            1         0            0   
4      Maruti     120000         130000            0         0            1   
...       ...        ...            ...          ...       ...          ...   
8123  Hyundai     110000         320000            0         0            1   
8124  Hyundai     119000         135000            1         0            0   
8125   Maruti     120000         382000            1         0            0   
8126     Tata      25000         290000            1         0            0   
8127     Tata      25000         290000            1         0            0   

      owner_Fourth & Above Owner  owner_Second Owner  owner_Test Drive Car  \
0                              0                   0                     0   
1                              0                   1                     0   
2                              0                   0                     0   
3                              0                   0                     0   
4                              0                   0                     0   
...                          ...                 ...                   ...   
8123                           0                   0                     0   
8124                           1                   0                     0   
8125                           0                   0                     0   
8126                           0                   0                     0   
8127                           0                   0                     0   

      owner_Third Owner  
0                     0  
1                     0  
2                     1  
3                     0  
4                     0  
...                 ...  
8123                  0  
8124                  0  
8125                  0  
8126                  0  
8127                  0  

[8128 rows x 10 columns]

## 3. OneHotEncoding using Sklearn

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,0:4],df.iloc[:,-1],test_size=0.2,random_state=2)

In [ ]:
X_train.head()

brand  km_driven    fuel         owner
5571   Hyundai      35000  Diesel   First Owner
2038      Jeep      60000  Diesel   First Owner
2957   Hyundai      25000  Petrol   First Owner
7618  Mahindra     130000  Diesel  Second Owner
6684   Hyundai     155000  Diesel   First Owner

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(drop='first',sparse=False,dtype=np.int32)

In [ ]:
X_train_new = ohe.fit_transform(X_train[['fuel','owner']])

In [ ]:
X_test_new = ohe.transform(X_test[['fuel','owner']])

In [ ]:
X_train_new.shape

(6502, 7)

In [ ]:
np.hstack((X_train[['brand','km_driven']].values,X_train_new))

array([['Hyundai', 35000, 1, ..., 0, 0, 0],
       ['Jeep', 60000, 1, ..., 0, 0, 0],
       ['Hyundai', 25000, 0, ..., 0, 0, 0],
       ...,
       ['Tata', 15000, 0, ..., 0, 0, 0],
       ['Maruti', 32500, 1, ..., 1, 0, 0],
       ['Isuzu', 121000, 1, ..., 0, 0, 0]], dtype=object)

## 4. OneHotEncoding with Top Categories

In [ ]:
counts = df['brand'].value_counts()

In [ ]:
df['brand'].nunique()
threshold = 100

In [ ]:
repl = counts[counts <= threshold].index

In [ ]:
pd.get_dummies(df['brand'].replace(repl, 'uncommon')).sample(5)

BMW  Chevrolet  Ford  Honda  Hyundai  Mahindra  Maruti  Renault  Skoda  \
8093    0          0     0      0        1         0       0        0      0   
3274    0          0     0      0        0         0       1        0      0   
2966    0          0     0      0        0         0       1        0      0   
1092    1          0     0      0        0         0       0        0      0   
5355    0          0     0      0        0         0       0        0      0   

      Tata  Toyota  Volkswagen  uncommon  
8093     0       0           0         0  
3274     0       0           0         0  
2966     0       0           0         0  
1092     0       0           0         0  
5355     0       0           0         1